In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime, timedelta

headers = {
    'User-Agent': 'Mozilla/5.0'
}

sources = {
    'Detik': 'https://www.detik.com/tag/',
    'Liputan6': 'https://www.liputan6.com/tag/',
    'Kompas': 'https://www.kompas.com/tag/'
}

kategori_list = {
    'Non-Sepak Bola': ['bulutangkis', 'basket', 'voli'],
    'Liga Inggris': ['liga-inggris'],
    'Liga Indonesia': ['liga-1'],
    'Liga Spanyol': ['la-liga'],
    'Liga Italia': ['serie-a'],
}

batas_waktu = datetime.now() - timedelta(days=180)

def proses_artikel(link, kategori):
    try:
        print(f"🔄 Mengambil artikel dari {link}")
        r_artikel = requests.get(link, headers=headers, timeout=10)
        r_artikel.raise_for_status()
        soup_artikel = BeautifulSoup(r_artikel.text, 'html.parser')

        # Ambil judul artikel
        judul = soup_artikel.title.get_text(strip=True) if soup_artikel.title else ''

        # Ambil isi teks artikel sesuai situs
        if 'detik.com' in link:
            paragraphs = soup_artikel.select('div.detail__body-content p')
        elif 'liputan6.com' in link:
            paragraphs = soup_artikel.select('div.article-content-body p')
        elif 'kompas.com' in link:
            paragraphs = soup_artikel.select('div.read__content p')
        else:
            paragraphs = []

        isi_teks = ' '.join(p.get_text(strip=True) for p in paragraphs)

        if len(isi_teks.split()) < 100:
            return None

        return {
            'Judul': judul,
            'Isi': isi_teks,
            'Url': link,
            'Label': kategori
        }
    except Exception as e:
        print(f"❗ Error mengambil {link}: {e}")
        return None

def scrape_berita():
    semua_berita = []
    for kategori, tag_list in kategori_list.items():
        for tag in tag_list:
            print(f"\n🔄 Mengambil berita dari tag: {tag} (Kategori: {kategori})")

            for sumber, base_url in sources.items():
                halaman = 1
                berita_per_kategori = 0
                max_berita = 1

                while halaman <= 1 and berita_per_kategori < max_berita:
                    url = f"{base_url}{tag}?page={halaman}"
                    try:
                        res = requests.get(url, headers=headers, timeout=10)
                        res.raise_for_status()
                    except Exception as e:
                        print(f"❗ Gagal akses halaman {url}: {e}")
                        break

                    soup = BeautifulSoup(res.text, 'html.parser')

                    if 'detik.com' in base_url:
                        links = [a['href'] for a in soup.select('article > a[href]') if a['href'].startswith('http')]
                    elif 'liputan6.com' in base_url:
                        links = [a['href'] for a in soup.select('div.tag-lists__wrap a[href]') if a['href'].startswith('http')]
                    elif 'kompas.com' in base_url:
                        links = [a['href'] for a in soup.select('div.article__list a.article__link[href]') if a['href'].startswith('http')]
                    else:
                        links = []

                    if not links:
                        links = [a['href'] for a in soup.select('a[href]') if a['href'].startswith('http')]

                    for link in links:
                        if link in [b['Url'] for b in semua_berita]:
                            continue

                        berita = proses_artikel(link, kategori)
                        if berita:
                            semua_berita.append(berita)
                            berita_per_kategori += 1
                            if berita_per_kategori >= max_berita:
                                break

                    halaman += 1
                    time.sleep(2)

    return semua_berita

# Ambil berita
semua_berita = scrape_berita()

# Simpan ke CSV dengan format kolom yang diminta
if semua_berita:
    df = pd.DataFrame(semua_berita)
    df.insert(0, 'No', range(1, len(df) + 1))  # Tambah kolom No dari 1 sampai jumlah data
    df.to_csv('berita_olahraga_scraped.csv', index=False, columns=['No', 'Judul', 'Isi', 'Url', 'Label'])
    print("\n📁 Data berhasil disimpan ke 'berita_olahraga_scraped.csv'")
else:
    print("\n❗ Tidak ada berita yang berhasil dikumpulkan.")



🔄 Mengambil berita dari tag: bulutangkis (Kategori: Non-Sepak Bola)
🔄 Mengambil artikel dari https://sport.detik.com/raket/d-7920224/rekap-final-thailand-open-2025
🔄 Mengambil artikel dari https://sport.detik.com/raket/d-7919041/jonatan-christie-antara-program-latihan-hingga-anggaran-turnamen
🔄 Mengambil artikel dari https://www.detik.com/jogja/berita/d-7918868/blak-blakan-jonatan-christie-ungkap-alasan-mundur-dari-pelatnas-pbsi
🔄 Mengambil artikel dari https://sport.detik.com/raket/d-7917849/ambisi-jonatan-christie-olimpiade-los-angeles-2028
🔄 Mengambil artikel dari https://sport.detik.com/raket/d-7917140/singapore-open-jadi-debut-jonatan-christie-sebagai-pemain-non-pelatnas
🔄 Mengambil artikel dari https://sport.detik.com/raket/d-7917117/soal-gregoria-taufik-hidayat-kita-doakan-sehat-cepat-main-lagi
🔄 Mengambil artikel dari https://sport.detik.com/fotosport/d-7745204/jojo-menangi-laga-pertama-indonesia-masters-2025
🔄 Mengambil artikel dari https://food.detik.com/foto-kuliner/d-74745

KeyboardInterrupt: 